In [46]:
import torch
import torch.nn as nn

In [183]:
C = 2
B_SZ = 1

### Normalize over each channel

**momentum** – the value used for the running_mean and running_var computation. Can be set to None for cumulative moving average (i.e. simple average). Default: 0.1

In [184]:
bn = nn.BatchNorm2d(C, momentum=1) # mom=1 makes it easier to calculate

In [186]:
x = torch.randn(B_SZ,C,3,3); x

tensor([[[[-2.3222,  1.9308,  2.1354],
          [-1.3982, -0.8408, -1.2045],
          [ 0.7950, -0.2078, -0.7867]],

         [[ 0.1609,  0.8470,  0.6291],
          [-0.5628, -0.3989, -2.2772],
          [ 1.8890,  0.5079, -0.8011]]]])

In [190]:
# batch, c, w, h
x.shape

torch.Size([1, 2, 3, 3])

In [191]:
y = bn(x)

In [192]:
x[:,0].mean(), x[:,1].mean()

(tensor(-0.2110), tensor(-0.0007))

In [193]:
x[:,0].var(), x[:,1].var()

(tensor(2.3358), tensor(1.4086))

In [194]:
bn.running_mean

tensor([-0.2110, -0.0007])

In [195]:
bn.running_var

tensor([2.3358, 1.4086])

In [196]:
y.mean(), y.var()

(tensor(-1.9868e-08, grad_fn=<MeanBackward0>),
 tensor(1.0588, grad_fn=<VarBackward0>))